In [97]:
import pandas as pd
import numpy as np
import logging
import os

pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None


In [98]:
from galaxy_datasets.pytorch.galaxy_datamodule import GalaxyDataModule

In [99]:
from sklearn.model_selection import train_test_split

In [100]:
from galaxy_datasets import gz2,gz_decals_5  # or gz_hubble, gz_candels, ...

gzd5_catalog_train, _ = gz_decals_5(
    root='data/gzd5',
    train=True,
    download=False
)

gzd5_catalog_test, _ = gz_decals_5(
    root='data/gzd5',
    train=False,
    download=False
)


In [101]:
gzd5_catalog = pd.concat([gzd5_catalog_train,gzd5_catalog_test])
gzd5_rings = pd.read_csv('data/target-mining/gzd5_rings.csv')
gzd5_not_rings = gzd5_catalog[~gzd5_catalog.file_loc.isin(gzd5_rings.file_loc_x.to_list())]

ring_part = gzd5_rings[['file_loc_x']]
ring_part.columns = ['file_loc']
ring_part['ring'] = 1

not_ring_part = gzd5_not_rings.sample(ring_part.shape[0]*10*3)[['file_loc']]
not_ring_part['ring'] = 0


In [111]:
not_ring_part_train, not_ring_part_test_val = train_test_split(not_ring_part, test_size = 0.3)
not_ring_part_test, not_ring_part_val = train_test_split(not_ring_part_test_val, test_size = 0.33)

ring_part_train, ring_part_test_val = train_test_split(ring_part, test_size = 0.3)
ring_part_test, ring_part_val = train_test_split(ring_part_test_val, test_size = 0.33)


ring_parts_train = []
ring_parts_test = []
ring_parts_val = []
for i in range(3):
    ring_parts_train.append(ring_part_train.sample(ring_part_train.shape[0],replace=True))
    ring_parts_test.append(ring_part_test.sample(ring_part_test.shape[0],replace=True))
    ring_parts_val.append(ring_part_val.sample(ring_part_val.shape[0],replace=True))
ring_part_train =  pd.concat(ring_parts_train)
ring_part_test =  pd.concat(ring_parts_test)
ring_part_val =  pd.concat(ring_parts_val)



In [112]:
train_set = pd.concat([not_ring_part_train,ring_part_train]).reset_index(drop=True).reset_index(names=['id_str'])
test_set = pd.concat([not_ring_part_test,ring_part_test]).reset_index(drop=True).reset_index(names=['id_str'])
val_set = pd.concat([not_ring_part_val,ring_part_val]).reset_index(drop=True).reset_index(names=['id_str'])


In [113]:
train_set.to_csv('train-test/gzd5_catalog_train.csv',index = False)
test_set.to_csv('train-test/gzd5_catalog_test.csv',index = False)
val_set.to_csv('train-test/gzd5_catalog_val.csv',index = False)

In [122]:
val_set.ring.mean()

0.0911336132378365

In [123]:
val_set

,id_str,file_loc,ring
0,0,data/gzd5/images/J100/J100053.73+170311.6.jpg,0
1,1,data/gzd5/images/J100/J100606.23+092621.0.jpg,0
2,2,data/gzd5/images/J124/J124218.89+103403.9.jpg,0
3,3,data/gzd5/images/J141/J141510.87+202831.5.jpg,0
4,4,data/gzd5/images/J090/J090517.08+301741.7.jpg,0
...,...,...,...
4044,4044,data/gzd5/images/J130/J130910.85+103701.7.jpg,1
4045,4045,data/gzd5/images/J130/J130736.35-024912.6.jpg,1
4046,4046,data/gzd5/images/J112/J112217.16+141039.2.jpg,1
4047,4047,data/gzd5/images/J143/J143053.53+111411.8.jpg,1


In [124]:
train_set

,id_str,file_loc,ring
0,0,data/gzd5/images/J165/J165340.47+175724.2.jpg,0
1,1,data/gzd5/images/J152/J152058.78+244239.9.jpg,0
2,2,data/gzd5/images/J105/J105040.19-011438.6.jpg,0
3,3,data/gzd5/images/J122/J122455.10+114215.9.jpg,0
4,4,data/gzd5/images/J090/J090700.45+170758.7.jpg,0
...,...,...,...
28615,28615,data/gzd5/images/J120/J120447.20-024312.2.jpg,1
28616,28616,data/gzd5/images/J114/J114552.90+101430.7.jpg,1
28617,28617,data/gzd5/images/J135/J135314.05+160956.3.jpg,1
28618,28618,data/gzd5/images/J143/J143419.01+033951.8.jpg,1


In [125]:
test_set

,id_str,file_loc,ring
0,0,data/gzd5/images/J114/J114605.87+074241.7.jpg,0
1,1,data/gzd5/images/J104/J104440.36+035627.8.jpg,0
2,2,data/gzd5/images/J150/J150911.26+205929.2.jpg,0
3,3,data/gzd5/images/J015/J015205.72-073002.3.jpg,0
4,4,data/gzd5/images/J031/J031455.94+000721.6.jpg,0
...,...,...,...
8213,8213,data/gzd5/images/J152/J152015.76+023918.3.jpg,1
8214,8214,data/gzd5/images/J154/J154033.45+170740.6.jpg,1
8215,8215,data/gzd5/images/J102/J102734.54+192705.0.jpg,1
8216,8216,data/gzd5/images/J105/J105333.55+192825.4.jpg,1
